# Haskell 101

Este notebook contiene el código de una charla dada el día 10 de diciembre de 2016. Para una introducción más extensa puedes leer [estos apuntes](https://github.com/libreim/haskell/blob/master/apuntes/introHaskell.pdf) así como los recursos que enlazan. Idea original y código de [@pedritomelenas](https://github.com/pedritomelenas), con comentarios y modificaciones de [@mx-psi](https://github.com/mx-psi).

## Introducción

Haskell es un lenguaje de programación funcional puro con tipos fuertes y estáticos y evaluación perezosa. Un programa en Haskell se basa en una serie de definiciones en forma de ecuaciones. El compilador o intérprete evaluará cada expresión sustituyendo el lado izquierdo de la ecuación por el derecho, sustituyendo los argumentos si es necesario.

Si **pruebas a descomentar la linea `a = 5`** obtendrás un error: las igualdades son definiciones y no asignaciones, y no podemos dar dos definiciones distintas de la misma cosa.

In [ ]:
a :: Int
a = 42
--a = 5

f :: Int -> Int
f x = x + 2

b = f a + f a + f a


Como ves las funciones se escriben separando con espacios sus argumentos. En un lenguaje de programación imperativo no podemos en general optimizar el cálculo de `b`: debemos **calcular 3 veces `f a`** ya que podría devolver un valor aleatorio, comunicarse con el mundo real o cambiar variables globales. 

En Haskell podemos [razonar algebraicamente](https://stackoverflow.com/questions/210835) sobre nuestros programas y el compilador puede realizar toda clase de optimizaciones. **Situa el cursor en la siguiente celda y presiona `Ctrl+Enter` para evaluar `b`**.

In [ ]:
b

## El sistema de tipos

Los tipos en Haskell son **fuertes** y **estáticos**. Se conoce el tipo de cada expresión en tiempo de compilación y los *castings* deben ser explícitos. Esto puede tener algunos inconvenientes, sin embargo los tipos son **inferidos**. 

A continuación algunas expresiones con tipos básicos como caracteres o cadenas de caracteres. El tipo de una expresión `expresion` se indica `expresion :: Tipo`. El nombre de los tipos empieza por mayúscula.

In [ ]:
haskell :: String
haskell = "haskell.org/platform"

lambda :: Char
lambda = 'λ' -- Permite Unicode

e :: Double
e = exp 1


El sistema de inferencia de tipos permite conocer el tipo de una expresión a partir de los tipos de las expresiones que lo constituyen. De esta forma el compilador puede deducir que la constante `b` definida anteriormente tiene tipo `Int`. **Puedes comprobar el tipo de `expresion` indicando `:t expresion`**:

In [ ]:
:t b

Expresar el tipo de cada función nos sirve a modo de documentación y lo haremos durante el resto de este notebook aunque no es necesario en la mayoría de los casos (puedes comprobarlo en el siguiente ejemplo que define la función `nand`). 

Cuando una función toma más de un argumento estos se separan por `->` (el por qué se explica en la sección de *currificación*).

In [ ]:
nand :: Bool -> Bool -> Bool
nand x y = not (x && y)


In [ ]:
:t nand True False

## Variables de tipo

Algunas funciones no tienen por qué reducirse a un sólo tipo: la función `id` (la identidad), definida por la ecuación `id x = x` podría tomar `Int` como entrada, pero también `Char`, `String` o cualquier otro tipo que definamos. 

En estos casos Haskell utiliza **variables de tipo**, que pueden sustituirse por cualquier tipo. Así la función `id` tiene tipo $\forall a: a \to a$, ya que toma algo de un tipo arbitario $a$ y devuelve algo del mismo tipo. Las variables de tipo se indican **en minúscula**.

In [ ]:
:t id

En general podemos tener un número arbitrario de variables de tipo independientes: la función `const`, que toma dos argumentos y devuelve el primero tiene tipo $\forall a \; \forall b: a \to b \to a$

_**Ejercicio**: Define la función `const`_

In [ ]:
-- Ya está definida, así que le ponemos otro nombre
-- La comilla es un caracter válido para los identificadores
const' :: a -> b -> a

## Patrones

Como hemos visto en las secciones anteriores podemos definir una función utilizando el esquema:
```haskell
f x1 x2 ... xn = ...
```
de tal forma que los argumentos quedan ligados a las variables `x1` a `xn`. Haskell permite también definir las funciones por casos en función del tipo que escojamos.

En el siguiente ejemplo definimos la función `niega :: Bool -> Bool` que define la negación lógica:

In [ ]:
niega True = False
niega False = True

Definimos la función para cada posible instancia del tipo. El compilador entonces evaluará en orden cada ecuación hasta que encuentre una en la que encaje. 

Podemos combinar las variables y los patrones como en el siguiente predicado que comprueba si un número es cero:

In [ ]:
esCero :: Int -> Bool
esCero 0 = True
esCero _ = False

El símbolo `_` es una convención que indica que no va a utilizarse ese argumento.

_**Ejercicio**: Define la función `factorial:: Int -> Int`_

In [ ]:
factorial :: Int -> Int
-- Necesitarás dos casos

In [ ]:
factorial 20

_**Ejercicio**: Define la función `(&&) :: Bool -> Bool -> Bool` (conjunción lógica)_

In [ ]:
-- Como esta función ya está definida le ponemos otro nombre:
y :: Bool -> Bool -> Bool

In [ ]:
-- Podemos llamar funciones de forma infija entre acentos graves
True `y` False

## Currificación

Como hemos visto antes, las funciones con más de un argumento tienen un tipo de la forma `A -> (B -> C)`, en lugar del tipo `A×B -> C`. Esto se debe a la [currificación](https://en.wikipedia.org/wiki/Currying), la identificación entre las funciones:

$$f : A \times B \to C \qquad f(a,b) = c$$
$$f : A \to (B \to C) \qquad g = f(a), g(b) = c$$

En lugar de tener una función que toma dos argumentos y devuelve un resultado, tenemos una función que toma un argumento y devuelve otra función que toma otro argumento para devolver el resultado. De esta forma podemos aplicar parcialmente funciones, como en el caso de la función `suma5`, que suma 5 a otra:

In [ ]:
suma5 = (5+)

In [ ]:
suma5 3

Aunque no es habitual quizás ayude ver un ejemplo equivalente en Python, definiendo de forma currificada una función que suma dos números:

```python
def suma(x):
  def sumax(y):
    return x + y
  return sumax

```

De esta forma `suma(2)` sería una función que suma 2, mientras que `suma(2)(3)` tendría el valor 5.

De esta forma podemos crear pequeñas funciones que componer para crear otras más complejas. Utilizando el operador `.` que compone funciones podemos crear por ejemplo una función que sume 12:

In [ ]:
suma7 =(7+)

In [ ]:
suma12  = suma5 . suma7

In [ ]:
suma12 9

_**Ejercicio**: Define una función que indique si un número no es cero haciendo uso de la composición._

In [ ]:
noEsCero :: Int -> Bool
noEsCero = _

_**Ejercicio**: ¿Cuál es el tipo de la función `.`? Escribe su tipo y compruébalo en la siguiente celda._

In [ ]:
:t (.)

## Listas

Las listas almacenan colecciones ordenadas de valores de un tipo homogéneo. Su tamaño no es fijo, es decir, listas de distintos tamaños pertenecen al mismo tipo.

Una lista se escribe entre corchetes, separando sus elementos por comas. La lista vacía se escribe `[]`:

In [ ]:
l1 :: [Int]
l1 = [1,2,3]

l2 = l1 ++ [4,5] -- Concatenar

Como vemos, indicamos en el tipo que se trata de una lista de enteros. El operador `++` permite concatenar dos listas, mientras que el operador `:` permite anteponer un elemento a otra lista:

In [ ]:
0 : l2

_**Ejercicio**: ¿Cuál es el tipo de la lista vacía? Compruébalo_

In [ ]:
:t []

Para definir funciones sobre listas utilizamos el reconocimiento de patrones. En este caso definimos una función que nos devuelve el primer elemento de una lista (si la lista es vacía devolverá un `error`). Utilizamos el constructor `:`, que como vimos antes antepone un elemento a una lista.

Esta función viene ya definida como `head`:

In [ ]:
cabeza :: [a] -> a
cabeza [] = error "Lista vacia"
cabeza (x:_) = x

In [ ]:
cabeza l2

_**Ejercicio:** Define una función que devuelva la `cola` de una lista_

In [ ]:
cola :: [a] -> [a]
cola [] = error "Lista vacia"
--- ...

Otro ejemplo es una función `longitud` que nos devuelva la longitud de una lista (finita). La longitud de una lista vacía es 0, mientras que la de una lista no vacía es uno más la longitud de su cola (la lista sin su primer elemento):

In [ ]:
longitud :: [a] -> Int
longitud [] = 0
longitud (_:xs) = 1 + longitud xs

Si queremos calcular la longitud de `l1` el compilador hará algo equivalente a:

```haskell
longitud [1,2,3]
1 + longitud [2,3]
1 + (1 + longitud [3])
1 + (1 + (1 + longitud []))
1 + (1 + (1 + 0))
1 + (1 + 1)
1 + 2
3
```
Esta función viene ya definida como `length` (aunque su tipo es más general ya que puede definirse también sobre otros tipos de estructuras que no sean listas):

In [ ]:
longitud l1

La función `(!!) :: [a] -> Int -> a` nos permite acceder al elemento en . Es una función infija: cuando indicamos su tipo o la pasamos como argumento la escribimos entre paréntesis pero cuando la definimos o utilizamos la podemos poner de forma infija.

La definimos aquí con 3 exclamaciones para no pisar la definición estandar:

In [ ]:
(!!!) :: [a] -> Int -> a
[]     !!! _ = error "Índice demasiado grande"
(x:_)  !!! 0 = x
(_:xs) !!! n = xs !!! (n-1)

In [ ]:
[1,2,3] !!! 1

Para llamarla de forma prefija le ponemos paréntesis:

In [ ]:
(!!!) [1,2,3] 0

_**Ejercicio**: Define la función `++` que concatena dos listas_

In [ ]:
concatena :: [a] -> [a] -> [a]
-- Tu definición

In [ ]:
[1,2] `concatena` [1,2] -- [1,2,1,2]

## Funciones de orden superior

Las listas son una parte fundamental de Haskell. Para trabajar con ellas suelen utilizarse **funciones de orden superior**: funciones que toman otras funciones como argumentos.

Un primer ejemplo es la función `map :: (a -> b) -> [a] -> [b]`, que toma una función y nos devuelve su versión sobre listas:

```
map f [a1, a2, ..., an] = [f a1, f a2, ..., f an]
```

Su definición es la siguiente (escribimos `map'` para no redefinir la función que ya está incluida):

In [ ]:
map' :: (a -> b) -> [a] -> [b]
map' f []     = []
map' f (x:xs) = f x : map' f xs

De esta forma y combinándolo con la aplicación parcial por la currificación podemos por ejemplo definir una función que tome una lista y sume 1 a todos sus elementos:

In [ ]:
suma1 = map (+1)
-- Equivalentemente: suma1 xs = map (+1) xs

In [ ]:
suma1 [1,2,3]

_**Ejercicio**: Define una función que tome una lista de listas y tome sus `cabezas`. Indica su tipo:_

In [ ]:
cabezas :: -- Indica su tipo
cabezas = -- Su definición

Otra función útil y más general es la función `foldr`, que toma una operación binaria y un valor por defecto y aplica esa operación a los elementos de una lista:
```haskell
foldr (⊕) z [a1,...,an] = a1 ⊕ (a2 ⊕ (... ⊕ (an ⊕ z)))
```

`foldr` puede utilizarse para definir cualquier función que siga el siguiente esquema:

```haskell
f [] = z
f (x:xs) = x ⊕ f xs
```

Un ejemplo común es la función `sum`, que suma los elementos de una lista y que podemos definir como:

```haskell
sum []     = 0
sum (x:xs) = x + sum xs
```

Utilizando `foldr` su definición se reduce a `sum = foldr (+) 0`:

In [ ]:
foldr (+) 0 [1,2,3]

Un caso útil de la función `foldr` es su aplicación a los [monoides](https://hackage.haskell.org/package/base-4.9.0.0/docs/Data-Monoid.html), reduciendo una lista de elementos del monoide a uno sólo mediante la aplicación de la operación del monoide.

_**Ejercicio**: Aprovechando la estructura de monoide de las funciones sobre un tipo define la función `compose :: [a -> a] -> a -> a` que compone una lista de funciones en una sola_

In [ ]:
compose :: [a -> a] -> a -> a
compose = -- Tu definición

In [ ]:
compose [(+1),(*2),(+3)] 0 -- (0 + 3)*2 + 1

_**Ejercicio**: Define las funciones `++`, `length` y `map` a partir de `foldr`_

Un último ejemplo de función de orden superior es `filter`, que filtra una lista según un predicado:

In [ ]:
filtro:: (a->Bool) -> [a] -> [a]
filtro f [] = []
filtro f (x:xs) = if f x then x:(filtro f xs) else filtro f xs

In [ ]:
filtro (>0) [1,-2,3]

## Definición de tipos 

Para definir un tipo se utiliza la palabra clave `data` y se definen distintos **constructores de datos** que empiezan por mayúscula y están separados por `|`. Estas funciones nos permiten crear instancias de ese tipo. Por ejemplo, el tipo `Bool` está definido:

```haskell
data Bool = False | True
```

De forma equivalente el tipo `Int` podría definirse listando todos los números (aunque en este caso la definición es primitiva):
```haskell
data Int = -9223372036854775808|...|-1|0|1|...|9223372036854775807
```

Cuando listamos los posibles casos de una función sólo podemos utilizar los constructores de datos para ello. Como primer ejemplo definimos un tipo `Forma` con 3 posibles instancias: triángulos,cuadrados y círculos y damos una función que nos indica si una forma es un polígono:

In [ ]:
data Forma = Triangulo | Cuadrado | Circulo -- :t Triangulo ?

esPoligono :: Forma -> Bool
esPoligono Triangulo = True
esPoligono Cuadrado  = True
esPoligono Circulo   = False

Los constructores de datos también pueden tomar argumentos, en cuyo caso indicamos el tipo de los mismos. `deriving Show` nos permite mostrar instancias de un tipo como cadenas de caracteres.

Definimos también una función que nos da la edad de una persona y otra que nos indica si dos personas tienen la misma edad:

In [ ]:
data Persona = P String Int deriving Show -- :t P ?

-- Obtener la edad de una persona
getEdad :: Persona -> Int
getEdad (P nombre edad) = edad

-- Tienen la misma edad?
mismaEdad :: Persona -> Persona -> Bool
mismaEdad (P _ e1) (P _ e2) = e1 == e2

In [ ]:
P "Pedro" 47

En general un tipo puede tener un número arbitrario de constructores de datos con cualquier número de argumentos.

Podemos definir los tipos de forma recursiva: un ejemplo clásico son los naturales definidos como en la aritmética de Peano: un natural puede ser el cero (`Z :: Nat`) o el sucesor de otro natural (`S :: Nat -> Nat` es la función sucesor).

Definimos también la suma de naturales y una función que transforma naturales a enteros, así como el valor `infinito`:

In [ ]:
-- Tipos recursivos
data Nat = Z | S Nat -- Naturales (Peano)

suma :: Nat -> Nat -> Nat
-- suma de naturales
suma   Z   n = n
suma (S n) m = S (suma n m)

toInt :: Nat -> Int
-- Pasar a entero
toInt   Z   = 0
toInt (S n) = 1 + toInt n

infinito = S infinito

In [ ]:
toInt (S (S Z))

_**Ejercicio**: Define mediante recursión mutua dos predicados que indiquen si un natural es `par` o `impar`_

### Constructores de tipos

Los **constructores de tipos** son funciones sobre tipos: toman uno o más tipos y devuelven un nuevo tipo. Ya hemos visto un caso concreto: el constructor de tipos lista `[]` que se definiría:

```haskell
data [a] = [] | a:[a] -- Sintaxis no legal
```

Es decir: una lista de tipo `a` puede ser la lista vacía o un elemento de tipo `a` antepuesto a una lista de tipo `a`. Otro ejemplo es el constructor de tipos `Either`, que nos permite crear un tipo que [*suma* otros dos](http://tux.ugr.es/dgiim/blog/2015/03/24/algebra-tipos):

```haskell
data Either a b = Left a | Right b
```

Como ejemplo final de esta sección definimos los árboles binarios de tipo `a` y una función que devuelve el reflejado de un árbol:

In [ ]:
data Tree a = Empty | Node a (Tree a) (Tree a)

-- Árbol reflejado
refl :: Tree a -> Tree a
refl Empty = Empty
refl (Node a t1 t2) = Node a (refl t2) (refl t1)


_**Ejercicio**: Define una función que devuelva la `altura :: Tree a -> Int` de un árbol_

## Clases de tipos

Algunos tipos tienen propiedades comunes. Para aprovecharlas
se definen **clases de tipos**, que agrupan tipos con la misma interfaz. La mayoría de los tipos son instancias de `Eq`, porque disponen de una función `(==)` que comprueba la igualdad. Las instancias de la clase `Num` pueden sumarse y multiplicarse, las de `Show` pueden convertirse a una `String` y las de `Ord` pueden ordenarse. 

Una clase de tipos se define a partir de las funciones que pueden utilizar de forma común los tipos que pertenecen a la misma. Por ejemplo, `Eq` se define:

```haskell
class  Eq a  where
    (==) :: a -> a -> Bool
    (/=) :: a -> a -> Bool
    -- Sólo hace falta definir uno, el otro por defecto:
    x /= y =  not (x == y)
    x == y =  not (x /= y)
```

Como ejemplo declaramos como instancia de la clase `Eq` al tipo de los naturales `Nat`:

In [ ]:
instance Eq Nat where
  Z   == Z   = True
  S n == S m = n == m
  _   ==  _  = False

In [ ]:
S Z == infinito

Otro ejemplo serían los árboles, pero en estos debemos imponer la restricción de que el tipo de sus elementos sea comparable por igualdad:

In [ ]:
instance (Eq a) => Eq (Tree a) where
  Empty        == Empty           = True
  Node a t1 t2 == Node a' t1' t2' = a == a' 
                                  && t1 == t1' && t2 == t2'

Cuando una función toma tipos que deben ser de una cierta clase se indica en su declaración de tipo con `=>`. Por ejemplo la función `elem` nos indica si un valor es elemento de una lista:

```haskell
elem :: (Eq a) => a -> [a] -> Bool
x `elem` []     = False
x `elem` (y:ys) = x == y || x `elem` ys
```

In [ ]:
1 `elem` [2,3,4]

## Functores

Por último definimos una clase de tipos sobre constructores de tipos de uso habitual: los functores.

Un functor `f` es un contenedor o un contexto computacional sobre el que podemos definir una función `fmap :: (Functor f) => (a -> b) -> f a -> f b` que aplique una función `a -> b` sobre todos los elementos de una estructura `f a` para devolver otra `f b`.

Un ejemplo de functores serían las listas, para las que `fmap` es la función `map`.

In [ ]:
fmap (++"!") ["Azúcar", "Especias", "Cosas bonitas"]

Toda instancia de la clase `Functor` debe cumplir las siguiente **leyes**:

```haskell
fmap id         == id
fmap f . fmap g == fmap (f . g)
```

Estas leyes vienen de la [teoría de categorías](https://en.wikipedia.org/wiki/Functor) y permiten realizar toda clase de optimizaciones al código escrito con functores.

Como último ejemplo definimos `Tree` como instancia de los functores:

In [ ]:
instance Functor Tree where
  fmap f Empty          = Empty
  fmap f (Node a t1 t2) = Node (f a) (fmap f t1) (fmap f t2)

# Para saber más

Si quieres **aprender más** puedes echar un vistazo a los recursos disponibles en [libreim/haskell](https://github.com/libreim/haskell#haskell).